In [ ]:
#yes
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt_tab')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
import string
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import chardet

data = pd.read_csv('/content/train.csv')
def preprocess_text(text):
    if not isinstance(text, str):
        return ''

    stop_words = set(stopwords.words('english'))
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)


data['cleaned_text'] = data['text'].apply(preprocess_text)



X = data['cleaned_text']
y = data['sentiment']


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=100)
X_test_padded = pad_sequences(X_test_seq, maxlen=100)
y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pickle

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# yes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense,Bidirectional
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
X = data['cleaned_text']
y = data['sentiment']
y = y.map(label_mapping)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_padded, y_train_onehot, epochs=10, batch_size=64, validation_data=(X_test_padded, y_test_onehot))


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


344/344 ━━━━━━━━━━━━━━━━━━━━ 158s 440ms/step - accuracy: 0.5228 - loss: 0.9545 - val_accuracy: 0.7031 - val_loss: 0.7176
Epoch 2/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 201s 440ms/step - accuracy: 0.7436 - loss: 0.6353 - val_accuracy: 0.7224 - val_loss: 0.6985
Epoch 3/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 189s 401ms/step - accuracy: 0.7854 - loss: 0.5511 - val_accuracy: 0.7113 - val_loss: 0.7119
Epoch 4/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 138s 401ms/step - accuracy: 0.8213 - loss: 0.4789 - val_accuracy: 0.6998 - val_loss: 0.7514
Epoch 5/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 142s 402ms/step - accuracy: 0.8481 - loss: 0.4162 - val_accuracy: 0.6982 - val_loss: 0.8242
Epoch 6/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 145s 422ms/step - accuracy: 0.8707 - loss: 0.3473 - val_accuracy: 0.6840 - val_loss: 0.9383
Epoch 7/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 196s 405ms/step - accuracy: 0.8898 - loss: 0.2985 - val_accuracy: 0.6802 - val_loss: 1.0461
Epoch 8/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 139s 405ms/step - accuracy: 0.9090 - loss: 0.24

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 100, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 100, 128)            │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,463,563 (9.40 MB)

 Trainable params: 821,187 (3.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,642,376 (6.27 MB)

In [ ]:
# yes
loss, accuracy = model.evaluate(X_test_padded, y_test_onehot, batch_size=64)


print(f'Test Accuracy: {accuracy * 100:.2f}%')


86/86 ━━━━━━━━━━━━━━━━━━━━ 11s 125ms/step - accuracy: 0.6886 - loss: 0.8397
Test Accuracy: 69.84%


In [ ]:
# Load test data
test_data = pd.read_csv('/content/test.csv')

# Preprocess text
test_data['cleaned_text'] = test_data['text'].apply(preprocess_text)
X_test_test=test_data['cleaned_text']
# Convert text to sequences
X_test_seq = tokenizer.texts_to_sequences(test_data['cleaned_text'])
X_test_padded_test = pad_sequences(X_test_seq, maxlen=100)


label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
y_test_test = test_data['sentiment'].map(label_mapping)
y_encoded_test = label_encoder.transform(test_data['sentiment'])
# One-hot encode the labels
y_test_onehot_test = to_categorical(y_encoded_test, num_classes=3)

# Evaluate the model



In [ ]:
    loss, accuracy = model.evaluate(X_test_padded_test, y_test_onehot_test, batch_size=64)

    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 145ms/step - accuracy: 0.6781 - loss: 1.5094
Test Loss: 1.5313
Test Accuracy: 67.15%


In [ ]:
import tensorflow as tf

model.save("sentiment_model.h5")
import joblib

# Save the model using joblib
joblib.dump(model, 'sentiment_model.pkl')

loaded_model = tf.keras.models.load_model("sentiment_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

converter._experimental_lower_tensor_list_ops = False

tflite_model = converter.convert()

with open("sentiment_model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp95yeb72g'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name='keras_tensor_63')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  133246417313360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133246417313168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133246417313552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133246417311440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133246417312592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133246417311632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133246417310288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133246417312784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133246417309520: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
file_size = len(tflite_model)  # Get size in bytes
file_size_kb = file_size / 1024
file_size_mb = file_size_kb / 1024

print(f"Size of TFLite model: {file_size} bytes ({file_size_kb:.2f} KB, {file_size_mb:.2f} MB)")


Size of TFLite model: 3304756 bytes (3227.30 KB, 3.15 MB)


In [ ]:
import os

file_size = os.path.getsize("sentiment_model.h5")
file_size_kb = file_size / 1024
file_size_mb = file_size_kb / 1024

print(f"Size of Keras model (HDF5): {file_size} bytes ({file_size_kb:.2f} KB, {file_size_mb:.2f} MB)")


Size of Keras model (HDF5): 9897040 bytes (9665.08 KB, 9.44 MB)


In [ ]:
!pip install tflite-runtime
!pip install ytmusicapi
from ytmusicapi import YTMusic

# Initialize YTMusic
ytmusic = YTMusic()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
import random

def get_spotify_recommendations(sentiment):
    if sentiment == 'positive':
        genre = 'pop'
    elif sentiment == 'negative':
        genre = 'sad'
    else:
        genre = 'classical'

    results = ytmusic.search(genre, filter="songs")

    random.shuffle(results)

    recommendations = []
    for song in results[:5]:
        title = song['title']
        artists = ", ".join(artist['name'] for artist in song['artists'])
        recommendations.append(f"{title} by {artists}")
    return recommendations


import numpy as np
import tensorflow as tf
correct_predictions = 0
total_predictions = len(X_test_padded_test)

interpreter = tf.lite.Interpreter(model_path='sentiment_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_data = np.array(X_test_padded_test, dtype=np.float32)

actual_labels = np.argmax(y_test_onehot_test, axis=1)
labels = ['neutral', 'positive', 'negative']

results = []

for i in range(total_predictions):
    sample = np.expand_dims(input_data[i], axis=0)
    interpreter.set_tensor(input_details[0]['index'], sample)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_details[0]['index'])
    predicted_index = np.argmax(predictions, axis=1)[0]
    predicted_label = labels[predicted_index]
    actual_label = labels[actual_labels[i]]
    # recommendations = get_spotify_recommendations(predicted_label)

    results.append({
        'input_text': X_test_test.iloc[i],
        'predicted_label': predicted_label,
        'actual_label': actual_label,
        'probabilities': predictions[0].tolist(),
        # 'recommendations': recommendations
    })
    if predicted_label == actual_label:
        correct_predictions += 1
for result in results:
    print(f"Input Text: {result['input_text']}")
    print(f"Predicted Label: {result['predicted_label']}")
    print(f"Actual Label: {result['actual_label']}")
    print(f"Probabilities: {result['probabilities']}")
    # print(f"Music Recommendations:")
    # for recommendation in result['recommendations']:
    #     print(f"  - {recommendation}")
    # print('-' * 50)

accuracy = (correct_predictions / total_predictions) * 100
print(f"Accuracy: {accuracy:.2f}%")


Streaming output truncated to the last 5000 lines.
Predicted Label: positive
Actual Label: neutral
Probabilities: [0.010103672742843628, 0.6893583536148071, 0.30053800344467163]
Input Text: heyy
Predicted Label: positive
Actual Label: neutral
Probabilities: [0.0015711047453805804, 0.9974562525749207, 0.0009727100259624422]
Input Text: radio royals transmitter packed today ... good anyone got spare shed ... lol
Predicted Label: positive
Actual Label: negative
Probabilities: [0.39493194222450256, 0.5948106646537781, 0.010257351212203503]
Input Text: lol wow good haha still believe mother day
Predicted Label: negative
Actual Label: positive
Probabilities: [0.00828761886805296, 0.027545921504497528, 0.9641664624214172]
Input Text: saw cute movie passed place called peasants kitchen wtf names kinda sad
Predicted Label: positive
Actual Label: neutral
Probabilities: [0.13995614647865295, 0.5262225866317749, 0.3338213264942169]
Input Text: across water damned expensive
Predicted Label: neutral

In [ ]:
X_test_padded[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0, 206, 601,  49], dtype=int32)

In [ ]:
import pandas as pd

# Assuming X_train_padded and y_train are already defined
X_df = pd.DataFrame(X_train_padded)

# Convert y_train to DataFrame and reshape to a column
y_df = pd.DataFrame(y_train, columns=['sentiment'])

# Concatenate X_train_padded and y_train
merged_df = pd.concat([X_df, y_df], axis=1)

# Remove rows where 'sentiment' or the first column (column 0) in X_df is null
merged_df = merged_df[(~merged_df['sentiment'].isnull()) & (~merged_df[0].isnull())]

# Save to CSV
merged_df.to_csv('merged_data.csv', index=False)
